In [286]:
import numpy as np
import pandas as pd
import os
from functools import reduce
from bamboo_lib.connectors.models import Connector
from bamboo_lib.models import EasyPipeline
from bamboo_lib.models import Parameter
from bamboo_lib.models import PipelineStep
from bamboo_lib.steps import DownloadStep
from bamboo_lib.steps import LoadStep
from bamboo_lib.helpers import grab_connector

CARPETAS_DICT = {
    1: "01 INFORMACIÓN INSTITUCIONAL",
    2: "02 CLIENTES ATENDIDOS",
    3: "03 SERVICIOS BRINDADOS",
    4: "04 PROYECTOS DE INVERSIÓN PÚBLICA",
    5: "05 EJECUCIÓN PRESUPUESTAL",
    6: "06 RECURSOS HUMANOS",
    7: "07 PARTIDAS ARANCELARIAS",
}


In [287]:
df = pd.read_excel('../../data/01. Información ITP red CITE  (01-10-2020)/07 PARTIDAS ARANCELARIAS/TABLA_08_N01 (18-10-2020).xlsx')

In [288]:
df

,cite,cadena_productiva,partida_arancelaria,descripcion_partida,tipo_exportación,sector
0,CITEproductivo Maynas,Aguaje,8.109090e+08,LOS DEMAS FRUTAS U OTROS FRUTOS FRESCOS,NO TRADICIONAL,AGROPECUARIO
1,CITEproductivo Madre de Dios,Aguaje,8.109090e+08,LOS DEMAS FRUTAS U OTROS FRUTOS FRESCOS,NO TRADICIONAL,AGROPECUARIO
2,UTagroindustrial Ambo,Aguaymanto,8.109050e+08,UCHUVAS (UVILLAS) (PHYSALIS PERUVIANA) FRESCAS,NO TRADICIONAL,AGROPECUARIO
3,CITEproductivo Madre de Dios,Aguaymanto,8.109050e+08,UCHUVAS (UVILLAS) (PHYSALIS PERUVIANA) FRESCAS,NO TRADICIONAL,AGROPECUARIO
4,CITEagroindustrial Oxapampa,Aguaymanto,8.109050e+08,UCHUVAS (UVILLAS) (PHYSALIS PERUVIANA) FRESCAS,NO TRADICIONAL,AGROPECUARIO
...,...,...,...,...,...,...
2023,UTagroindustrial Ambo,Yuca,7.141000e+08,"RAÍCES DE YUCA (MANDIOCA) FRESCAS, REFRIGERADA...",NO TRADICIONAL,AGROPECUARIO
2024,UTagroindustrial Ambo,Yuca,1.901200e+09,MEZCLAS Y PASTAS PARA LA PREPARACIÓN DE PRODUC...,NO TRADICIONAL,AGROPECUARIO
2025,UTagroindustrial Ambo,Yuca,1.106209e+09,"HARINA, SÉMOLA Y POLVO DE SAGU O DE LAS RAÍCES...",NO TRADICIONAL,AGROPECUARIO
2026,UTagroindustrial Ambo,Yuca,2.008999e+09,DEMÁS FRUTAS O FRUTOS Y DEMÁS PARTÍCULAS COMES...,NO TRADICIONAL,AGROPECUARIO


In [289]:
df['partida_arancelaria']

0       8.109090e+08
1       8.109090e+08
2       8.109050e+08
3       8.109050e+08
4       8.109050e+08
            ...     
2023    7.141000e+08
2024    1.901200e+09
2025    1.106209e+09
2026    2.008999e+09
2027    1.108140e+09
Name: partida_arancelaria, Length: 2028, dtype: float64

In [291]:
df = df[df['cite'].notna()]
df = df.rename(columns={'descripcion_partida ' : 'descripcion_partida','partida_arancelaria' : 'partida_id','tipo_exportación' : 'tipo_exportacion'})
df['descripcion_partida'] = df['descripcion_partida'].str.capitalize()
df['tipo_exportacion'] = df['tipo_exportacion'].str.capitalize()
df['sector'] = df['sector'].str.capitalize()
df['partida_id'] = df['partida_id'].astype(str).str[:-6].str.zfill(6)
df['cantidad_cite'] = 1
df['cadena_productiva'] = df['cadena_productiva'].str.strip()

In [292]:
## shared dim

cite_list = list(df["cite"].unique())
cite_map = {k:v for (k,v) in zip(sorted(cite_list), list(range(1, len(cite_list) +1)))}
df['cite_id'] = df['cite'].map(cite_map)

In [293]:
## short dim

sector_list = list(df["sector"].unique())
sector_map = {k:v for (k,v) in zip(sorted(sector_list), list(range(1, len(sector_list) +1)))}
df['sector_id'] = df['sector'].map(sector_map)

In [294]:
## short dim

tipo_exp_list = list(df["tipo_exportacion"].unique())
tipo_exp_map = {k:v for (k,v) in zip(sorted(tipo_exp_list), list(range(1, len(tipo_exp_list) +1)))}
df['tipo_exp_id'] = df['tipo_exportacion'].map(tipo_exp_map)

In [295]:
## long dim

cadena_productiva_list = list(df["cadena_productiva"].unique())
cadena_productiva_map = {k:v for (k,v) in zip(sorted(cadena_productiva_list), list(range(1, len(cadena_productiva_list) +1)))}
df['cad_prod_id'] = df['cadena_productiva'].map(cadena_productiva_map)

In [296]:
df = df[['cad_prod_id','cadena_productiva']]

In [297]:
df.drop_duplicates()

,cad_prod_id,cadena_productiva
0,1,Aguaje
2,2,Aguaymanto
5,3,Ajo
8,4,Ají charapita
18,5,Ají limo
...,...,...
1991,87,Trucha
2000,88,Uva
2014,89,Yacón
2018,90,Yogurth


In [298]:
df

,cad_prod_id,cadena_productiva
0,1,Aguaje
1,1,Aguaje
2,2,Aguaymanto
3,2,Aguaymanto
4,2,Aguaymanto
...,...,...
2023,91,Yuca
2024,91,Yuca
2025,91,Yuca
2026,91,Yuca


In [299]:
pivot = df[['cite_id','sector_id','cad_prod_id','partida_id','tipo_exp_id','cantidad_cite']]

KeyError: "['partida_id', 'tipo_exp_id', 'sector_id', 'cantidad_cite', 'cite_id'] not in index"

In [262]:
pivot

,cite_id,sector_id,cad_prod_id,partida_id,tipo_exp_id,cantidad_cite
0,21,1,1,081090,1,1
1,20,1,1,081090,1,1
2,25,1,2,081090,1,1
3,20,1,2,081090,1,1
4,7,1,2,081090,1,1
...,...,...,...,...,...,...
2023,25,1,92,071410,1,1
2024,25,1,92,190120,1,1
2025,25,1,92,110620,1,1
2026,25,1,92,200899,1,1


In [263]:
dim2 = df[['descripcion_partida','partida_id']]

In [264]:
dim2 = dim2.drop_duplicates()

In [265]:
dim2[dim2.duplicated()]

,descripcion_partida,partida_id


In [266]:
dim2

,descripcion_partida,partida_id
0,Los demas frutas u otros frutos frescos,081090
2,Uchuvas (uvillas) (physalis peruviana) frescas,081090
5,"Ajos, frescos o refrigerados",070320
6,Ajos para siembra,070320
7,"Ajos secos, cortados en trozos o rodajas, trit...",071290
...,...,...
2018,"Los demás yogur, excepto los aromatizados con ...",040310
2020,"Yogur aromatizado con frutas, cacao u otros fr...",040310
2023,"Raíces de yuca (mandioca) frescas, refrigerada...",071410
2024,Mezclas y pastas para la preparación de produc...,190120


In [267]:
        k = 1
        df = {}
        for i in range(1,1 +1):
            path, dirs, files = next(os.walk("../../data/01. Información ITP red CITE  (01-10-2020)/{}/".format(CARPETAS_DICT[i])))
            file_count = len(files)


            for j in [1,5,6,7]:
                file_dir = "../../data/01. Información ITP red CITE  (01-10-2020)/{}/TABLA_0{}_N0{}.csv".format(CARPETAS_DICT[i],i,j)

                df[k] = pd.read_csv(file_dir)
                k = k + 1
        
        df_list = [df[i] for i in range(1,4)]
        df = reduce(lambda df1,df2: pd.merge(df1,df2,on=['cite'],how='outer'), df_list)

        cadenas = df[['cadena_resolucion','cadena_pip','cadena_atencion']]

        cadena_resolucion_list = list(cadenas["cadena_resolucion"].unique())
        cadena_resolución_map = {k:v for (k,v) in zip(sorted(cadena_resolucion_list), list(range(1, len(cadena_resolucion_list) +1)))}
        
        cadena_pip_list = list(cadenas["cadena_pip"].unique())
        cadena_pip_map = {k:v for (k,v) in zip(sorted(cadena_pip_list), list(range(1, len(cadena_pip_list) +1)))}

        cadena_atencion_list = list(cadenas["cadena_atencion"].unique())
        cadena_atencion_map = {k:v for (k,v) in zip(sorted(cadena_atencion_list), list(range(1, len(cadena_atencion_list) +1)))}
        
        cadenas['cadena_resolucion_id'] = cadenas['cadena_resolucion'].map(cadena_resolución_map).astype(str)
        cadenas['cadena_pip_id'] = cadenas['cadena_pip'].map(cadena_pip_map).astype(str)
        cadenas['cadena_atencion_id'] = cadenas['cadena_atencion'].map(cadena_atencion_map).astype(str)
        
        cadenas['cadena_resolucion_id_temp'] = cadenas['cadena_resolucion_id'].str.zfill(2)
        cadenas['cadena_pip_id_temp'] = cadenas['cadena_pip_id'].str.zfill(2)
        cadenas['cadena_atencion_id_temp'] = cadenas['cadena_atencion_id'].str.zfill(2)

        cadenas['cadena_resolucion_id'] = (cadenas['cadena_resolucion_id_temp']).astype(int) 
        cadenas['cadena_pip_id'] = (cadenas['cadena_resolucion_id_temp'] + cadenas['cadena_pip_id_temp']).astype(int)  
        cadenas['cadena_atencion_id']  = (cadenas['cadena_pip_id'].astype(str)  + cadenas['cadena_atencion_id_temp']).astype(int)  

        df = cadenas[['cadena_resolucion','cadena_resolucion_id','cadena_pip','cadena_pip_id','cadena_atencion','cadena_atencion_id']]



In [268]:
df

,cadena_resolucion,cadena_resolucion_id,cadena_pip,cadena_pip_id,cadena_atencion,cadena_atencion_id
0,Acuícola,1,Acuícola,102,"Tilapia, paiche, paco, gamitana y doncella",10217
1,Agroindustrial,2,Frutas y hortalizas,210,"Frutas y hortalizas, derivados lácteos",21010
2,Café y frutas de la zona,5,"Café, cacao y frutas (piña)",505,"Café, cacao y frutas de la zona",50503
3,Agroindustrial,2,"Frutos, hortalizas, menestras y granos andinos",213,"Uva, espárrago, cebolla, manzana, higo, pláta...",21319
4,Agroindustrial,2,"Frutas, hortalizas y hierbas aromáticas",211,"Uva, hongos comestibles, derivados lácteos, ma...",21120
5,Agroindustrial,2,"Uva, palta y orégano",218,"Uva, palta y orégano",21821
6,Agroindustrial,2,Granadilla y palta,214,"Café, cacao, aguaymanto, granadilla, quito qui...",21404
7,Café y cacao,4,Café y cacao,404,Café y cacao,40402
8,Cuero calzado,8,Cuero y calzado,808,Cuero y calzado,80808
9,"Cuero, calzado e industrias conexas",10,Cuero y calzado e industrias conexas,1009,Cuero y calzado,100908
